## Part 1

In [1]:
import re
def is_consistent(arrangement: str, criteria: list[int]):
    contiguous_groups = [len(x) for x in re.findall(r'#+', arrangement)]
    return contiguous_groups == criteria

In [2]:
import itertools

def generate_arrangements(record: str):
    num_missing = record.count('?')
    return (record.replace('?', '{}').format(*arrangement) for arrangement in itertools.product('#.', repeat=num_missing))

In [3]:
def count_arrangements_naive(record: str, criteria: list[int]):
    return sum(is_consistent(arrangement, criteria) for arrangement in generate_arrangements(record))

In [4]:
with open('input.txt')  as f:
    lines = f.read().splitlines()

In [5]:
def count_arrangements(record: str, criteria: list[int]):
    result = 0

    if '#' not in record and criteria == []:
        # print("No more groups, no more broken springs: consistent")
        # print('\t', record, criteria, '\n')
        return 1 # even if we have lots of '?' remaining, the only consistent arrangement is when they are all '.', given that there are no more groups

    if (record.count('?') + record.count('#') < sum(criteria)) or (record.count('#') > sum(criteria)):
        # print("Too many definitely or too few possibly broken springs: inconsistent")
        # print('\t', record, criteria, '\n')
        return 0 # if there are too few or too many springs, it's not consistent

    if record[0] in '?.': # if the first spring is not faulty, we can just skip it
        # print("Assume start is not faulty")
        # print('\t', record, criteria, '\n')
        result += count_arrangements(record[1:], criteria)

    if record[0] in '?#':
        # print("Assume start is faulty")
        if len(record) >= criteria[0] and '.' not in record[:criteria[0]] and (len(record) == criteria[0] or record[criteria[0]] in '?.'): # order is important here for short-circuiting otherwise we get index out of range
            # print("Conditions met for first group to be faulty")
            # print('\t', record, criteria, '\n')
            result += count_arrangements(record[criteria[0] + 1:], criteria[1:]) # if it's a '.' after, we can skip it. if it's a '?', it has to be a '.', so we can skip it
        else:
            # print("Conditions not met for first group to be faulty")
            # print('\t', record, criteria, '\n')
            pass
    
    return result

In [6]:
num_arrangements = 0
for line in lines:
    record, criteria = line.split()
    criteria = [int(x) for x in criteria.split(',')]
    num_arrangements += count_arrangements(record, criteria)
print(num_arrangements)

8270


## Part 2

In [7]:
def unfold(record: str, criteria: list[int]):
    return '?'.join([record] * 5), criteria * 5
unfolded_records = [unfold(record, tuple([int(x) for x in criteria.split(',')])) for record, criteria in (line.split() for line in lines)]

In [8]:
cache = {}

def count_arrangements_cache(record: str, criteria: tuple[int]):
    result = 0

    if '#' not in record and not criteria:
        return 1 # even if we have lots of '?' remaining, the only consistent arrangement is when they are all '.', given that there are no more groups

    if (record.count('?') + record.count('#') < sum(criteria)) or (record.count('#') > sum(criteria)):
        return 0 # if there are too few or too many springs, it's not consistent
    
    if (record, criteria) in cache:
        return cache[(record, criteria)]

    if record[0] in '?.': # if the first spring is not faulty, we can just skip it
        result += count_arrangements_cache(record[1:], criteria)

    if record[0] in '?#':
        if criteria[0] <= len(record) and '.' not in record[:criteria[0]] and (criteria[0] == len(record) or record[criteria[0]] in '?.'): # order is important here for short-circuiting otherwise we get index out of range
            result += count_arrangements_cache(record[criteria[0] + 1:], criteria[1:]) # if it's a '.' after, we can skip it. if it's a '?', it has to be a '.', so we can skip it

    cache[(record, criteria)] = result
    return result

In [9]:
num_arrangements = 0
for record, criteria in unfolded_records:
    num_arrangements += count_arrangements_cache(record, criteria)
print(num_arrangements)

204640299929836
